# PyTorch training loop and Testing loop

For the training loop, we'll build the following steps:

1. Forward pass
2. Calculate the loss
3. 	Zero gradients
4. Perform backpropagation on the loss
5. Update the optimizer (gradient descent)

In [ ]:
import torch
import numpy as np
import seaborn as sns
from torch import nn

torch.manual_seed(42)

X = np.linspace(0,20,num=200)
y = X + np.sin(X)*2 + np.random.normal(size=X.shape)

In [ ]:
sns.scatterplot(x=X,y=y,color='Blue', label='Data')

In [ ]:
from torch.utils.data import Dataset , DataLoader

class Simple1DRegressionDataset(Dataset):
    def __init__(self,X,y):
        super(Simple1DRegressionDataset,self).__init__()
        self.X = X.reshape(-1,1)
        self.y = y.reshape(-1,1)

    def __getitem__(self,index):
        return torch.tensor(self.X[index,:],dtype=torch.float32) , torch.tensor(self.y[index,:],dtype=torch.float32)

    def __len__(self):
        return self.X.shape[0]

In [ ]:
df = Simple1DRegressionDataset(X,y)

In [ ]:
training_loader = DataLoader(Simple1DRegressionDataset(X,y), shuffle=True)

In [ ]:
loss_fn = nn.MSELoss()

In [ ]:
# so we need now a training loop
from tqdm.autonotebook import tqdm

def train_simple_network(model,
                        loss_fn,
                        training_loader,
                        optimizer,
                        epochs=5,
                        device='cpu'
                        ):


    model.to(device)

    for epoch in tqdm(range(epochs), desc="Epochs"):

        model = model.train()

        running_loss = 0

        for inputs , labels in tqdm(training_loader,desc='Batches'):

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            y_pred = model(inputs)

            loss = loss_fn(y_pred, labels)

            loss.backward()

            optimizer.step()

            #running_loss += loss.item()
            #print(running_loss)



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class regressionModel(nn.Module):
  def __init__(self):
      super().__init__()
      self.l1 = nn.Linear(1,32)
      self.l2 = nn.Linear(32,64)
      self.l3 = nn.Linear(64,128)
      self.l4 = nn.Linear(128,256)
      self.l5 = nn.Linear(256,1)
      self.activation = nn.ReLU()

  def forward(self, x):
    x = self.activation(self.l1(x))
    x = self.activation(self.l2(x))
    x = self.activation(self.l3(x))
    x = self.activation(self.l4(x))
    x = self.l5(x)
    return x


In [ ]:
model_3 = regressionModel()
optimizer3 = torch.optim.SGD(model_3.parameters(), lr=.001)
train_simple_network(model_3,
                loss_fn,
                training_loader,
                device=device,
                epochs=100,
                optimizer=optimizer3
                )


In [ ]:
# lets try our model
model_3.to('cpu')

model_3.eval()

with torch.inference_mode():
    y_pred_m3 = model_3(torch.tensor(X,dtype=torch.float32).reshape(-1,1)).cpu().numpy()

In [ ]:
sns.scatterplot(x=X,y=y , color='blue',label='Data')
sns.lineplot(x=X,y=y_pred_m3.ravel(),color='red',label='Linear model')
